In [1]:
!pip install cupy-cuda101

In [2]:
import torch
from google.colab import drive
drive.mount('/content/gdrive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

Mounted at /content/gdrive
1.7.0+cu101
cuda


In [6]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np
import cupy as cp

# data I/O
data = open('gdrive/My Drive/doupo_small.txt', 'r', encoding='utf-8').read()  # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

test = [char_to_ix[ch] for ch in data]

# hyperparameters
hidden_size = 100  # size of hidden layer of neurons
seq_length = 25  # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = cp.random.randn(hidden_size, vocab_size) * 0.01  # input to hidden
Whh = cp.random.randn(hidden_size, hidden_size) * 0.01  # hidden to hidden
Why = cp.random.randn(vocab_size, hidden_size) * 0.01  # hidden to output
bh = cp.zeros((hidden_size, 1))  # hidden bias
by = cp.zeros((vocab_size, 1))  # output bias


def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = cp.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = cp.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = cp.tanh(cp.dot(Wxh, xs[t]) + cp.dot(Whh, hs[t - 1]) + bh)  # hidden state
        ys[t] = cp.dot(Why, hs[t]) + by  # unnormalized log probabilities for next chars
        ps[t] = cp.exp(ys[t]) / cp.sum(cp.exp(ys[t]))  # probabilities for next chars
        loss += -cp.log(ps[t][targets[t], 0])  # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = cp.zeros_like(Wxh), cp.zeros_like(Whh), cp.zeros_like(Why)
    dbh, dby = cp.zeros_like(bh), cp.zeros_like(by)
    dhnext = cp.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = cp.copy(ps[t])
        dy[targets[t]] -= 1  # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += cp.dot(dy, hs[t].T)
        dby += dy
        dh = cp.dot(Why.T, dy) + dhnext  # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += cp.dot(dhraw, xs[t].T)
        dWhh += cp.dot(dhraw, hs[t - 1].T)
        dhnext = cp.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        cp.clip(dparam, -5, 5, out=dparam)  # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]


def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    x = cp.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = cp.tanh(cp.dot(Wxh, x) + cp.dot(Whh, h) + bh)
        y = cp.dot(Why, h) + by
        p = cp.exp(y) / cp.sum(cp.exp(y))
        ix = cp.random.choice(range(vocab_size), size=1, p=p.ravel())
        x = cp.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


n, p = 0, 0
mWxh, mWhh, mWhy = cp.zeros_like(Wxh), cp.zeros_like(Whh), cp.zeros_like(Why)
mbh, mby = cp.zeros_like(bh), cp.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -cp.log(1.0 / vocab_size) * seq_length  # loss at iteration 0
count = 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(data) or n == 0:
      hprev = cp.zeros((hidden_size, 1))  # reset RNN memory
      p = 0  # go from start of data
      count += 1
      if count > 30:
        break

    inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 100)
        sample_y = cp.asnumpy(sample_ix)
        y = [ix_to_char[int(ix)] for ix in sample_y]
        txt = ''.join(y)
        print('----\n %s \n----' % (txt,))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update

    p += seq_length  # move data pointer
    n += 1  # iteration counter

test_data = open('gdrive/My Drive/doupo_test_small.txt', 'r', encoding='utf-8').read()  # should be simple plain text file
xs, hs, ys, ps = {}, {}, {}, {}
hs[-1] = cp.zeros((hidden_size, 1))
for t in range(len(test)):
    xs[t] = cp.zeros((vocab_size, 1))  # encode in 1-of-k representation
    xs[t][test[t]] = 1
    hs[t] = cp.tanh(cp.dot(Wxh, xs[t]) + cp.dot(Whh, hs[t - 1]) + bh)  # hidden state
    ys[t] = cp.dot(Why, hs[t]) + by  # unnormalized log probabilities for next chars

pre = sample(hs[len(test)-1], test[-1],800)
out = [ix_to_char[int(ix)] for ix in pre]
print(test_data, end="")
print(''.join(out))

流式输出内容被截断，只能显示最后 5000 行内容。
    …

 
----
iter 103700, loss: 69.373453
----
 长输大的喝眉，斗篷的测验字，薰儿轻声道：“更是？

    望着面前面上巧笑，笑着金色大厅，有起拖着嘲讽，每次老头道。

    “小时候啊？被纳兰嫣然截历？”洒密的脸颊上，让得起脸色气氛，含魄的美脸 
----
iter 103800, loss: 69.330862
----
 谅，不过，萧炎的选择而球，这条审视，淡淡的笑了摆体轻的声音同后，萧炎抬头移缩了小脑袋，却依旧是有些苦涩的与筑基灵液，可有如此灵魂感径直把玩韧分，可萧炎哥哥也自精神来的，日威力的变，简直凶化当年争，却是 
----
iter 103900, loss: 69.437689
----
    见到萧炎哥哥，你相信防少圾分，干涩打误整的萧炎，迅速的斗技，那没有利义超灵大，那排不争，她过后，云岚宗还有事，我至内家族的族长，不仅清集消大小脸，却是见过那卷家族，就以坊花的首饰，一种护罩看这老 
----
iter 104000, loss: 69.931654
----
 “十万金币购买上，一双指身前，一双纤满脸后着淡淡的笑道。

    喜笑着隐隐盖了一片之内，不由方爱不断忽然无有严潮的霎回，这所还有人小脸蛋，是股缓缓藏了能支道，萧炎现在的小嘴中的加雾似得小福上气得常 
----
iter 104100, loss: 70.105629
----
 奥，屁有两名金星。

    “我买年时，可众境的，悻上次岁了！”轻注的道：“他半这家伙聚气散，这是在自己流帮手，便是加列奥颤酒逛，子一瓶，都没有相信的话，又是保持着客厉的个挣誓，阶斗师，那可不愿说？ 
----
iter 104200, loss: 70.083607
----
 小家荡着你失失的模样，断了，一片莫名的韵鉴宝室”下去，不过没想到四百金光中，虽然并不是没什么？”

    摊了揉淡淡的声音的薰儿，狠狠的门对，那熏儿手掌不是摇头，让得萧炎摸了出来，微微一些都未有着遗 
----
iter 104300, loss: 69.993518
----
 掉性，每天刺成了这三年，或者莫名。摊上，少年目光中的白色佣兵男，紧在而坐了椅的小脸毫带洗畅。

    听着药老，突如回复属消嘲，面前点是瞟首那